# Migrate

How to use: 
- Set files path with variable `PATH`
- Check file result and verify data
- Create Instalacions from xls file indicated with variable `FILENAME`

In [1]:
import pandas as pd
import re
from tqdm import tqdm
from erppeek import Client
PATH = '/home/puig/Arranque Medidas RE (EDM)/'
FILENAME = 'Generadors_tipo_3_i_5_(RE12-RE).xlsx'
UPR_FILENAME = 'UPRSL_0189_20180201.54'
host = 'http://edm-erp.clients:28069'
user = 'edm_distri'
db = 'root'
#c = Client(host, db, user)
c = Client('http://chilla.clients:8069', 'chilla', 'root')

error: [Errno 110] Connection timed out

In [183]:
def merge_instalacions():
    cols = [
        'cp_upr', 'cif', 'representant', 'codi_representant', 'Ud Repre', 
        'desc', 'ree_code', 'x', 'y', 'z', 'k', 'l', 'y'
    ]
    df_upr = pd.read_csv(PATH + UPR_FILENAME, delimiter=';', names=cols)
    df_upr = df_upr.groupby(['representant', 'cif', 'Ud Repre']).aggregate({'desc': 'count'}).reset_index()
    df_instalacions = pd.read_excel(PATH + FILENAME, dtype={'titular': str}, converters={u'Pérdidas': percent_lows})
    df_instalacions = df_instalacions.merge(df_upr, how='left', on='Ud Repre')
    
    return df_instalacions

In [4]:
def get_province_from_cups(cups_id):
    """get ree province code from cups id"""
    cups = c.model('giscedata.cups.ps').browse(cups_id)
    return cups.id_provincia.ree_code

In [5]:
def percent_lows(low):
    return low * 100

# Verify file content

In [332]:
pd.read_excel(PATH + FILENAME, converters={u'Pérdidas': percent_lows}).sample(2)

,cups,comptador,titular,cups_direcció,tarifa,potencia contractada,Tipus,Ud Repre,Pérdidas
97,ES0189000048088503YF1F,ZIV00039344170,DAVID VALLES SANCHO,CL POLÍGONO 9 PARC. 130-149 (1) 44640 (Torreci...,RE,10.0,5,FYEFG,0.0
54,ES0189000035088502JC1F,ZIV00044545070,"CUNICOLA LOS PEDREGALES, S.L.",CL POLIGONO 32 PARC. 49 44564 (Mas de las Matas),RE,9.9,5,FYNEXU,0.0


# Create instalacions functions

In [334]:
def create_instalacions(data, c):
    """Create a Instalacions RE"""
    TIPOS = [('1', 'Tipo 1'),('2', 'Tipo 2'),('3', 'Tipo 3'),('4', 'Tipo 4'),('5', 'Tipo 5')]
    UNITATS_LECTURA = {'kWh': 1, 'Wh': 1000}
    
    uprs_obj = c.model('giscedata.re.uprs')
    re_obj = c.model('giscedata.re')
    cups_obj = c.model('giscedata.cups.ps')
    partner_obj = c.model('res.partner')
    instalacio_values = []
    upr_values = []
    undone = []
    df_upr = pd.read_csv(PATH + UPR_FILENAME, 
            delimiter=';', 
            names=[
                'cp_upr', 'cif', 'representant', 'codi_representant', 'unitat_programacio', 'desc', 'ree_code', 
                'x', 'y', 'z', 'k', 'l', 'y'
            ])
    
    for inst in tqdm(data.values()):
        cups_id = cups_obj.search([('name', '=', inst['cups'])])[0]
        tipo = inst['Tipus']
        provincia =  get_province_from_cups(cups_id)
        unitat_programacio = inst['Ud Repre']
        name_grouped = (re.sub("\(.*\)", "", inst['titular'])).strip()
        partner_id = partner_obj.search([('name', '=', name_grouped)])
        
        if not partner_id:
            undone.append({'cups': inst['cups'], 'name': inst['titular']})
            continue
        
        partner_id = partner_id[0]
        
        representant = inst['representant'] + '%'
        representant_id = partner_obj.search([('name', '=like', representant)])

        # Notes: 
        # - el nom de la instalació será el nom del titular.
        # - cap camp de la poteècia és obligatori.
        # - la potencia nominal serà la potència contractada del excel.
        
        instalacio_values.append({
            'name': inst['titular'], 
            'cups': cups_id, 
            'titular': partner_id, 
            'provincia': provincia, 
            'tipo': str(tipo), 
            'unidad_lec': UNITATS_LECTURA['kWh'], 
            'perdidas': inst[u'P\xe9rdidas'], 
            'p_ae': False, 
            'p_as': False, 
            'active': True, 
            'potencia_nominal': inst['potencia contractada'],
        })
        
        # Create instalacio
        instalacio = re_obj.create(instalacio_values[-1])
        instalacio_id = instalacio.id
        
        upr_values.append({
            'name': unitat_programacio, 
            'representante': representant_id[0],
            'inst_re': instalacio_id,
            'data_alta': '2000-01-01 00:00:00',
            'data_baixa': '3000-01-01 00:00:00' # any 3000
        })
        
        # Create UPR
        unitat_programacio = uprs_obj.create(upr_values[-1])
        unitat_programacio_id = unitat_programacio.id
        
        # Assign UPR
        instalacio.write({'uprs': [(6, 0, [unitat_programacio_id])]})
        
    return pd.DataFrame(instalacio_values), pd.DataFrame(upr_values), pd.DataFrame(undone)

# Create installations

In [337]:
# Parse file and exctract data
df_instalacions = merge_instalacions()
_dict = df_instalacions.T.to_dict()
data = create_instalacions(_dict, c)

100%|██████████| 3/3 [00:05<00:00,  1.83s/it]


In [338]:
data[2]

""


In [147]:
data[0].loc[(data[0]['titular'] == False, ['cups', 'name', 'titular'])]

,cups,name,titular
12,59863,"CATI RENOVABLES, C.B. - II",False
85,59774,"PALAU MILIAN, S.L (9,2)",False
94,59864,"QUEROL GUARDIOLA, S.L.",False


- El primer no ho troba perque és `CATI RENOVABLES, C.B.` (l'_script_ només suporta noms repetits diferenciats pel contingut d'un parèntesi final)
- El segon i el tercer no els troba ja que els noms estan incomplerts:
    - al segon li falta un punt final.
    - al tercer li falta una U i un punt al final.

In [148]:
data[1][:2]

,data_alta,data_baixa,inst_re,name,representante
0,2000-01-01 00:00:00,3000-01-01 00:00:00,17,FYEEXXI,64437
1,2000-01-01 00:00:00,3000-01-01 00:00:00,-1,FYEEXXI,64417


# Crear fitxer xml

In [2]:
def gen_doc(di, df):
    tg_obj = c.model('tg.profile')
    cups = 'ES0189000014088503RX1F'
    p_id = c.model('giscedata.polissa').search([('cups', '=', cups)])
    m_id = c.model('giscedata.lectures.comptador').search([('polissa', '=', p_id)])
    m_data = c.model('giscedata.lectures.comptador').read(m_id[0], ['concentrator', 'meter_tg_name'])
    root = ET.Element("Report", IdRpt="S02", IdPet="749963711", Version="3.1.c")
    doc = ET.SubElement(root, "Cnc", Id=m_data['concentrator'])                        
    subdoc = ET.SubElement(doc, "Cnc", Id=m_data['meter_tg_name'], Magn="1")
    tg_ids = tg_obj.search([('name', '=', m_data['meter_tg_name']), ('timestamp', '>=', di), ('timestamp', '<=', df)])
    for x in tg_obj.read(tg_ids, ['ai', 'ae', 'r1', 'r2', 'r3', 'r4', 'bc', 'timestamp']):
        ET.SubElement(subdoc, "S02", Fh=timestamp_to_xml_fh(x['timestamp']), AE=str(x['ae']), AI=str(x['ai']), R1=str(x['r1']), R2=str(x['r2']), Bc=x['bc'])
    tree = ET.ElementTree(root)
    tree.write("filename.xml")

# Crear curves

In [7]:
from erppeek import Client
cprod = Client('http://morella-erp.clients:8069', 'maestrazgo_distribucion', 'root')
cpreprod = Client('http://edm-erp.clients:28069', 'edm_distri', 'root')

Password for 'root': ········
Password for 'root': ········


In [68]:
def get_registers_from_curve(tg, ree_contract_id, pol_id, curva_id, measures, arrastres):
    re_obj = cpreprod.model('giscedata.re')
    upr_obj = cpreprod.model('giscedata.re.uprs')
    agcl_obj = cpreprod.model('giscedata.perfils.agcl')
    
    year, month, day = tg['timestamp'][:10].split('-')
    hour = tg['timestamp'][11:13]
    name = year + month + day + hour
    
    ree_contract = re_obj.read(ree_contract_id, ['provincia', 'cups', 'uprs', 'tipo'])
    upr_name = upr_obj.read(ree_contract['uprs'][0], ['name'])
    uid = 1
    distribuidora = agcl_obj.default_distribuidora(uid)
    
    """
    lectura_real: (lectura * magnitud) / 1000 (W -> kW)
    perdues: perdues del contracte per cada magnitud (generalment 0)
    lectura: int(lectura) (truncada)
    arrastre: arrastre_anterior += part_decimal_lectura_real
    """
    res = []
    _tmp_vals = {
        'name': name, 'year': year, 'month': month, 'day': day, 
        'cups': ree_contract['cups'][1], 'provincia': ree_contract['provincia'], 'tipo': ree_contract['tipo'],
        'hour': int(hour), 'upr': upr_name, 'estacio': 1, 'distribuidora': distribuidora,
        'curva': curva_id
    }
    perdues_config = 0
    perdues = dict.fromkeys(measures, 0)
    for measure in measures:
        vals = _tmp_vals.copy()
        vals['magnitud'] = measure
        lectura = (tg['ai']  * tg['magn']) / 1000
        vals['lectura_real'] = lectura
        vals['perdues'] = perdues[measure] * perdues_config
        lectura -= lectura * vals['perdues']
        lectura += arrastres[measure]
        aprox = int(abs(round(lectura)))
        arrastres[measure] = lectura - aprox
        if abs(arrastres[measure]) > 1:
            aprox += int(arrastres[measure])
            arrastres[magnitud] -= int(arrastres[measure])
        vals['arrastre'] = arrastres[measure]
        vals['lectura'] = aprox
        #vals['estacio']
        res.append(vals)
    return res
    # TODO: get_estacio
    # TODO: comprovador d'hores totals!!!

In [65]:
# ae (active export) del model tg.profile = as (activa saliente)
# ai (active input) del model tg.profile = ae (activa entrante)

from datetime import datetime
def create_curve(di, df):
    pol_obj = cpreprod.model('giscedata.polissa')
    meter_obj = cpreprod.model('giscedata.lectures.comptador')
    prof_obj = cprod.model('tg.profile')
    re_curva_obj = cpreprod.model('giscedata.re.curva')
    re_curva_lectura_obj = cpreprod.model('giscedata.re.curva.lectura')
    cups = 'ES0189000014088503RX1F'
    ree_contract_id = 167
    p_id = pol_obj.search([('cups', '=', cups)])
    m_id = meter_obj.search([('polissa', '=', p_id)])
    meter_name = meter_obj.build_name_tg(m_id)
    tg_ids = prof_obj.search([('name', '=', meter_name), ('timestamp', '>=', di), ('timestamp', '<=', df)])
    
    vals = {
        'name': 'Curva creada el: {}'.format(datetime.now().strftime('%d/%m/%Y %H:%M:%S')),
        'inici': di,
        'final': df,
        'inst_re': ree_contract_id,
        'compact': True
    }
    curva_id = re_curva_obj.create(vals)
    measures = ['AE', 'AS', 'R1', 'R2', 'R3', 'R4']
    arrastres = dict.fromkeys(measures, 0)
    to_check = []
    for x, tg in enumerate(prof_obj.read(tg_ids, ['ai', 'ae', 'r1', 'r2', 'r3', 'r4', 'timestamp', 'magn'])):
        a = get_registers_from_curve(tg, ree_contract_id, p_id, curva_id, measures, arrastres)
        for elem in a:
            print '{} \n'.format(elem)
            to_check.append(elem)
            #re_curva_lectura_obj.create(elem)
    
    # Validator hours
    """diff = datetime.strptime(df, '%Y-%m-%d %H:%M:%S') - datetime.strptime(di, '%Y-%m-%d %H:%M:%S')
    total = (diff.seconds) / 3600 * 6
    assert total == len(to_check), 'total {} vs check {}'.format(total, len(to_check))"""

In [67]:
create_curve('2018-07-01 01:00:00', '2018-08-01 00:00:00')

{'lectura': 0, 'provincia': 'CS', 'name': '2018080100', 'hour': 0, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AE', 'month': '08', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.001, 'year': '2018', 'lectura_real': 0.001, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '01', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018080100', 'hour': 0, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AS', 'month': '08', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.001, 'year': '2018', 'lectura_real': 0.001, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '01', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018080100', 'hour': 0, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'R1', 'month': '08', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.001, 'year': '2018', 'lectura_real': 0.001, 'cups': 'ES0189000014088503RX1F', 'upr':

{'lectura': 0, 'provincia': 'CS', 'name': '2018073120', 'hour': 20, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AE', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '31', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018073120', 'hour': 20, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AS', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '31', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018073120', 'hour': 20, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'R1', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'

{'lectura': 0, 'provincia': 'CS', 'name': '2018073116', 'hour': 16, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AE', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '31', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018073116', 'hour': 16, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AS', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '31', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018073116', 'hour': 16, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'R1', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'

{'lectura': 0, 'provincia': 'CS', 'name': '2018073112', 'hour': 12, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AE', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '31', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018073112', 'hour': 12, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'AS', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'name': 'FYGAC', 'id': 161}, 'day': '31', 'tipo': '5'} 

{'lectura': 0, 'provincia': 'CS', 'name': '2018073112', 'hour': 12, 'perdues': 0, 'distribuidora': '0189', 'magnitud': 'R1', 'month': '07', 'curva': <Record 'giscedata.re.curva,40'>, 'estacio': 1, 'arrastre': 0.003, 'year': '2018', 'lectura_real': 0.0, 'cups': 'ES0189000014088503RX1F', 'upr': {'

KeyboardInterrupt: 